In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import csv

In [ ]:
def read_csv(filename, delimiter):
    """Reads a csv file and returns the data as lists"""

    timestamp_columnname = '__time'

    #define column names from ros topic names
    faro_x_columnname = '/faro_pos/pose/position/x'
    faro_y_columnname = '/faro_pos/pose/position/y'

    marv_x_columnname = '/transformed_pos/pose/pose/position/x'
    marv_y_columnname = '/transformed_pos/pose/pose/position/y'

    #read csv file to dataframe
    csv_dataframe = pd.read_csv(filename, delimiter=delimiter)

    #read columns from dataframe
    marv_x_data = csv_dataframe[marv_x_columnname]
    marv_y_data = csv_dataframe[marv_y_columnname]

    faro_x_data = csv_dataframe[faro_x_columnname]
    faro_y_data = csv_dataframe[faro_y_columnname]

    #remove NaNs
    marv_x_data = marv_x_data[np.logical_not(np.isnan(marv_x_data))]
    marv_y_data = marv_y_data[np.logical_not(np.isnan(marv_y_data))]

    faro_x_data = faro_x_data[np.logical_not(np.isnan(faro_x_data))]
    faro_y_data = faro_y_data[np.logical_not(np.isnan(faro_y_data))]

    return marv_x_data, marv_y_data, faro_x_data, faro_y_data

In [ ]:
pointslist = []

#batch read csv, create list of all points
for filename in os.listdir():
    if len(filename) <= 6: #short name files
        [marv_x,marv_y,faro_x,faro_y] = read_csv(filename, ',')
        marv_x = marv_x.to_list()
        marv_y = marv_y.to_list()
        faro_x = faro_x.to_list()
        faro_y = faro_y.to_list()
        pointslist.append([marv_x,marv_y,faro_x,faro_y])

In [ ]:
# plot all points for overview
ax = plt.subplot()

for point in pointslist:
    ax.scatter(point[0],point[1],c='b',marker='.') # x and y for marvelmind
    ax.scatter(point[2],point[3],c='r',marker='.') # x and y for faro
ax.set(xlabel='x (m)', ylabel='y (m)', title='statische Punkte')
ax.legend(['marv', 'faro'])
ax.grid() 

In [ ]:
#calculate mean and standard deviation for each point, then plot

#initialize mean value lists
marv_mean = [2,1]
faro_mean = [2,1]

fig,ax = plt.subplots()
iterator = 0

mean_devs = []
max_devs = []

for point in pointslist: #for all static points
    
    #use numpy to calculate mean for each points x and y coordinate seperately
    marv_mean[0]=np.mean(point[0])  #marvelmind x
    marv_mean[1]=np.mean(point[1])  #marvelmind y
    faro_mean[0]=np.mean(point[2])  #faro x
    faro_mean[1]=np.mean(point[3])  #faro y

    #use numpy to calculate standard deviation for each points x and y coordinate seperately
    marv_x_stddev = np.std(point[0])    #marvelmind x
    marv_y_stddev = np.std(point[1])    #marvelmind y
    faro_x_stddev = np.std(point[2])    #faro x
    faro_y_stddev = np.std(point[3])    #faro y

    deviations = []
    #calculate euclidean distance between mean and each measurement
    for i in range(len(point[0])):
        onex = point[0][i]
        oney = point[1][i]
        one = [onex,oney]   #combine to vector to use numpy linalg function
        deviations.append(np.linalg.norm(np.array(one)-np.array(marv_mean)))

    #get max and min deviation
    maxdev = np.max(deviations)
    meandev = np.mean(deviations)

    max_devs.append(maxdev)
    mean_devs.append(meandev)

    #plot circle with radius of max deviation
    ax.add_artist(plt.Circle([marv_mean[0],marv_mean[1]],maxdev, fill = False, ec = 'b'))

    #plot mean points
    ax.scatter(marv_mean[0],marv_mean[1],c='b',marker='+')

    #plot point labels
    ax.text(marv_mean[0]-0.1,marv_mean[1]-0.1, str(iterator))

    #plot raw points
    ax.scatter(point[0],point[1],c='b',marker='.',alpha=0.5)

    iterator += 1

ax.set(xlabel='x (m)', ylabel='y (m)', title='Statische Punkte')
ax.legend(['max', 'mittelwert','messwerte'])
ax.grid()

#print numeric values
print('mean deviations:')
print(mean_devs)

print('maximum deviations:')
print(max_devs)

print('mean of maximum deviations:')
print(np.mean(max_devs))

#write calculated values to csv
with open('staticpoint_results.csv', 'w', newline='') as csvfile:
    header = ['mean_devs','max_devs']
    filewriter = csv.writer(csvfile, delimiter=',')
    filewriter.writerow(header)
    rows = zip(mean_devs,max_devs)
    for row in rows:
        filewriter.writerow(row)